<a href="https://colab.research.google.com/github/battleship0000/NLP---online-shopping-chat-bot/blob/main/nlp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk spacy scikit-learn pandas --quiet

import pandas as pd
import re, string, nltk, spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


nlp = spacy.load("en_core_web_sm")


data = {
    "Intent": [
        "Order Status",
        "Return Policy",
        "Product Info",
        "Delivery Time",
        "Cancel Order",
        "Warranty",
        "Payment Issue",
        "Exchange Policy",
        "Order Help",
        "Feedback"
    ],
    "Example Query": [
        "Where is my order #12345?",
        "How can I return a product?",
        "Does this phone support fast charging?",
        "When will my order arrive?",
        "I want to cancel my order",
        "Does this product come with warranty?",
        "My payment failed but money was deducted",
        "
        ?",
        "I need help with my recent order",
        "I want to share my feedback"
    ],
    "Response": [
        "Your order {order_number} is currently being processed or out for delivery.",
        "You can return products within 15 days via our online portal.",
        "Yes, this product supports fast charging and comes with a one-year warranty.",
        "Orders are usually delivered within 3-5 business days.",
        "Your cancellation request has been received. Please check your email for confirmation.",
        "All products come with a 1-year manufacturer warranty unless stated otherwise.",
        "If your payment failed but amount was deducted, please contact support@shop.com with transaction ID.",
        "You can exchange products within 7 days for a different color or size.",
        "Sure! Please provide your order number so I can assist you better.",
        "We value your feedback! Please share it at feedback@shop.com."
    ]
}

faq_data = pd.DataFrame(data)


orders_db = {
    "#12345": {"product": "iPhone 15", "status": "Out for delivery", "delivery_date": "Nov 6, 2025"},
    "#67890": {"product": "Nike Air Shoes", "status": "Delivered", "delivery_date": "Nov 1, 2025"},
    "#64321": {"product": "Samsung TV", "status": "Processing", "delivery_date": "Nov 8, 2025"}
}


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = nltk.word_tokenize(text)
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(clean_tokens)

faq_data["cleaned_query"] = faq_data["Example Query"].apply(preprocess_text)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(faq_data["cleaned_query"])


def extract_order_number(text):
    match = re.findall(r"#\d+", text)
    return match[0] if match else None

def extract_product_name(text):
    doc = nlp(text)
    products = [ent.text for ent in doc.ents if ent.label_ in ["PRODUCT"]]
    return products[0] if products else None


def classify_intent(user_query):
    cleaned_query = preprocess_text(user_query)
    user_tfidf = vectorizer.transform([cleaned_query])
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix)
    best_match_index = similarity_scores.argmax()
    best_score = similarity_scores[0][best_match_index]

    if best_score < 0.3:
        return None, None
    intent = faq_data.iloc[best_match_index]["Intent"]
    response = faq_data.iloc[best_match_index]["Response"]
    return intent, response


def chatbot_response(user_query):
    order_number = extract_order_number(user_query)
    intent, response_template = classify_intent(user_query)

    if not intent:
        return "I'm sorry, I didn’t quite understand that. Could you rephrase or contact support@shop.com?"


    if intent == "Order Status":
        if order_number and order_number in orders_db:
            info = orders_db[order_number]
            return (f"📦 Order {order_number} Details:\n"
                    f"Product: {info['product']}\n"
                    f"Status: {info['status']}\n"
                    f"Expected Delivery: {info['delivery_date']}")
        elif order_number:
            return f"Sorry, I couldn't find details for order {order_number}."
        else:
            return "Please provide your order number (e.g., #12345) to check order status."


    if "{order_number}" in response_template and order_number:
        response_template = response_template.replace("{order_number}", order_number)

    return response_template


print("🤖 Welcome to the Smart Customer Support Chatbot!")
user_name = input("Please enter your name: ")
print(f"Hello {user_name}, how can I help you today? (Type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print(f"Chatbot: Goodbye {user_name}! 👋")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)